In [1]:
import pandas as pd #pandasをpdとしてインポート
import re
from tqdm import tqdm
tqdm.pandas()
 
#データをdfに読み込み。pandasをpdとして利用。
df_locations = pd.read_csv("csvs/latest.csv")
df_zips =  pd.read_csv("csvs/postalcode_roman.csv", header=None, dtype=str)
df_zips2 = df_zips.copy()
df_zips2.columns = ['郵便番号', '都道府県名2', '市区町村名2', '大字町丁目名2', '都道府県名カナ2', '市区町村名カナ2', '大字町丁目名カナ2']

In [2]:
df_locations

,都道府県コード,都道府県名,都道府県名カナ,都道府県名ローマ字,市区町村コード,市区町村名,市区町村名カナ,市区町村名ローマ字,大字町丁目名,大字町丁目名カナ,大字町丁目名ローマ字,小字・通称名,緯度,経度
0,1,北海道,ホッカイドウ,HOKKAIDO,1101,札幌市中央区,サッポロシチュウオウク,SAPPORO SHI CHUO KU,旭ケ丘一丁目,アサヒガオカ 1,ASAHIGAOKA 1,NaN,43.042230,141.319722
1,1,北海道,ホッカイドウ,HOKKAIDO,1101,札幌市中央区,サッポロシチュウオウク,SAPPORO SHI CHUO KU,旭ケ丘二丁目,アサヒガオカ 2,ASAHIGAOKA 2,NaN,43.039768,141.321733
2,1,北海道,ホッカイドウ,HOKKAIDO,1101,札幌市中央区,サッポロシチュウオウク,SAPPORO SHI CHUO KU,旭ケ丘三丁目,アサヒガオカ 3,ASAHIGAOKA 3,NaN,43.039569,141.319617
3,1,北海道,ホッカイドウ,HOKKAIDO,1101,札幌市中央区,サッポロシチュウオウク,SAPPORO SHI CHUO KU,旭ケ丘四丁目,アサヒガオカ 4,ASAHIGAOKA 4,NaN,43.038819,141.323040
4,1,北海道,ホッカイドウ,HOKKAIDO,1101,札幌市中央区,サッポロシチュウオウク,SAPPORO SHI CHUO KU,旭ケ丘五丁目,アサヒガオカ 5,ASAHIGAOKA 5,NaN,43.036547,141.322217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277186,47,沖縄県,オキナワケン,OKINAWA KEN,47215,南城市,ナンジョウシ,NANJO SHI,知念,チネンアザマ,CHINEN AZAMA,知名,26.183835,127.813337
277187,47,沖縄県,オキナワケン,OKINAWA KEN,47215,南城市,ナンジョウシ,NANJO SHI,知念,チネンアザマ,CHINEN AZAMA,吉富,26.167796,127.817767
277188,47,沖縄県,オキナワケン,OKINAWA KEN,47215,南城市,ナンジョウシ,NANJO SHI,知念,チネンアザマ,CHINEN AZAMA,久手堅,26.170679,127.823115
277189,47,沖縄県,オキナワケン,OKINAWA KEN,47215,南城市,ナンジョウシ,NANJO SHI,知念,チネンアザマ,CHINEN AZAMA,安座真,26.177382,127.823061


In [3]:
df_zips2

,郵便番号,都道府県名2,市区町村名2,大字町丁目名2,都道府県名カナ2,市区町村名カナ2,大字町丁目名カナ2
0,0600000,北海道,札幌市 中央区,以下に掲載がない場合,HOKKAIDO,SAPPORO SHI CHUO KU,IKANIKEISAIGANAIBAAI
1,0640941,北海道,札幌市 中央区,旭ケ丘,HOKKAIDO,SAPPORO SHI CHUO KU,ASAHIGAOKA
2,0600041,北海道,札幌市 中央区,大通東,HOKKAIDO,SAPPORO SHI CHUO KU,ODORIHIGASHI
3,0600042,北海道,札幌市 中央区,大通西（１〜１９丁目）,HOKKAIDO,SAPPORO SHI CHUO KU,ODORINISHI(1-19-CHOME)
4,0640820,北海道,札幌市 中央区,大通西（２０〜２８丁目）,HOKKAIDO,SAPPORO SHI CHUO KU,ODORINISHI(20-28-CHOME)
...,...,...,...,...,...,...,...
124536,9071433,沖縄県,八重山郡 竹富町,南風見仲,OKINAWA KEN,YAEYAMA GUN TAKETOMI CHO,HAIMINAKA
124537,9071751,沖縄県,八重山郡 竹富町,波照間,OKINAWA KEN,YAEYAMA GUN TAKETOMI CHO,HATERUMA
124538,9071544,沖縄県,八重山郡 竹富町,鳩間,OKINAWA KEN,YAEYAMA GUN TAKETOMI CHO,HATOMA
124539,9071800,沖縄県,八重山郡 与那国町,以下に掲載がない場合,OKINAWA KEN,YAEYAMA GUN YONAGUNI CHO,IKANIKEISAIGANAIBAAI


In [4]:
df_locations.loc[0]

都道府県コード                         1
都道府県名                         北海道
都道府県名カナ                    ホッカイドウ
都道府県名ローマ字                HOKKAIDO
市区町村コード                      1101
市区町村名                      札幌市中央区
市区町村名カナ               サッポロシチュウオウク
市区町村名ローマ字     SAPPORO SHI CHUO KU
大字町丁目名                     旭ケ丘一丁目
大字町丁目名カナ                 アサヒガオカ 1
大字町丁目名ローマ字           ASAHIGAOKA 1
小字・通称名                        NaN
緯度                       43.04223
経度                     141.319722
Name: 0, dtype: object

In [5]:
df_zips2.loc[0]

郵便番号                      0600000
都道府県名2                        北海道
市区町村名2                    札幌市　中央区
大字町丁目名2                以下に掲載がない場合
都道府県名カナ2                 HOKKAIDO
市区町村名カナ2      SAPPORO SHI CHUO KU
大字町丁目名カナ2    IKANIKEISAIGANAIBAAI
Name: 0, dtype: object

In [6]:
len(df_zips2.index)

124541

In [7]:
len(df_locations.index)

277191

In [8]:
df_zips2['都道府県名2'].str.cat(df_zips2['市区町村名2']).str.cat(df_zips2['大字町丁目名2']).str.replace('　', '')  # 全角スペースを削除

0           北海道札幌市中央区以下に掲載がない場合
1                  北海道札幌市中央区旭ケ丘
2                  北海道札幌市中央区大通東
3          北海道札幌市中央区大通西（１〜１９丁目）
4         北海道札幌市中央区大通西（２０〜２８丁目）
                  ...          
124536           沖縄県八重山郡竹富町南風見仲
124537            沖縄県八重山郡竹富町波照間
124538             沖縄県八重山郡竹富町鳩間
124539    沖縄県八重山郡与那国町以下に掲載がない場合
124540           沖縄県八重山郡与那国町与那国
Name: 都道府県名2, Length: 124541, dtype: object

In [9]:
df_zips2['full_name'] = df_zips2['都道府県名2'].str.cat(df_zips2['市区町村名2']).str.cat(df_zips2['大字町丁目名2']).str.replace('　', '')
df_zips2.head(3)

,郵便番号,都道府県名2,市区町村名2,大字町丁目名2,都道府県名カナ2,市区町村名カナ2,大字町丁目名カナ2,full_name
0,0600000,北海道,札幌市 中央区,以下に掲載がない場合,HOKKAIDO,SAPPORO SHI CHUO KU,IKANIKEISAIGANAIBAAI,北海道札幌市中央区以下に掲載がない場合
1,0640941,北海道,札幌市 中央区,旭ケ丘,HOKKAIDO,SAPPORO SHI CHUO KU,ASAHIGAOKA,北海道札幌市中央区旭ケ丘
2,0600041,北海道,札幌市 中央区,大通東,HOKKAIDO,SAPPORO SHI CHUO KU,ODORIHIGASHI,北海道札幌市中央区大通東


In [10]:
df_locations["full_name"] = df_locations["都道府県名"].str.cat(df_locations["市区町村名"]).str.cat(df_locations["大字町丁目名"])
df_locations.head(7)

,都道府県コード,都道府県名,都道府県名カナ,都道府県名ローマ字,市区町村コード,市区町村名,市区町村名カナ,市区町村名ローマ字,大字町丁目名,大字町丁目名カナ,大字町丁目名ローマ字,小字・通称名,緯度,経度,full_name
0,1,北海道,ホッカイドウ,HOKKAIDO,1101,札幌市中央区,サッポロシチュウオウク,SAPPORO SHI CHUO KU,旭ケ丘一丁目,アサヒガオカ 1,ASAHIGAOKA 1,NaN,43.042230,141.319722,北海道札幌市中央区旭ケ丘一丁目
1,1,北海道,ホッカイドウ,HOKKAIDO,1101,札幌市中央区,サッポロシチュウオウク,SAPPORO SHI CHUO KU,旭ケ丘二丁目,アサヒガオカ 2,ASAHIGAOKA 2,NaN,43.039768,141.321733,北海道札幌市中央区旭ケ丘二丁目
2,1,北海道,ホッカイドウ,HOKKAIDO,1101,札幌市中央区,サッポロシチュウオウク,SAPPORO SHI CHUO KU,旭ケ丘三丁目,アサヒガオカ 3,ASAHIGAOKA 3,NaN,43.039569,141.319617,北海道札幌市中央区旭ケ丘三丁目
3,1,北海道,ホッカイドウ,HOKKAIDO,1101,札幌市中央区,サッポロシチュウオウク,SAPPORO SHI CHUO KU,旭ケ丘四丁目,アサヒガオカ 4,ASAHIGAOKA 4,NaN,43.038819,141.323040,北海道札幌市中央区旭ケ丘四丁目
4,1,北海道,ホッカイドウ,HOKKAIDO,1101,札幌市中央区,サッポロシチュウオウク,SAPPORO SHI CHUO KU,旭ケ丘五丁目,アサヒガオカ 5,ASAHIGAOKA 5,NaN,43.036547,141.322217,北海道札幌市中央区旭ケ丘五丁目
5,1,北海道,ホッカイドウ,HOKKAIDO,1101,札幌市中央区,サッポロシチュウオウク,SAPPORO SHI CHUO KU,旭ケ丘六丁目,アサヒガオカ 6,ASAHIGAOKA 6,NaN,43.037676,141.318922,北海道札幌市中央区旭ケ丘六丁目
6,1,北海道,ホッカイドウ,HOKKAIDO,1101,札幌市中央区,サッポロシチュウオウク,SAPPORO SHI CHUO KU,大通西十丁目,オオドオリニシ 10,ODORINISHI 10,NaN,43.059131,141.341914,北海道札幌市中央区大通西十丁目


In [11]:
df_locations2 =  df_locations.loc[:,['full_name', '緯度', '経度']]
df_locations2.head(5)

,full_name,緯度,経度
0,北海道札幌市中央区旭ケ丘一丁目,43.042230,141.319722
1,北海道札幌市中央区旭ケ丘二丁目,43.039768,141.321733
2,北海道札幌市中央区旭ケ丘三丁目,43.039569,141.319617
3,北海道札幌市中央区旭ケ丘四丁目,43.038819,141.323040
4,北海道札幌市中央区旭ケ丘五丁目,43.036547,141.322217


In [12]:
df_zips2 =  df_zips2.loc[:,['郵便番号', 'full_name']]
df_zips2.head(5)

,郵便番号,full_name
0,0600000,北海道札幌市中央区以下に掲載がない場合
1,0640941,北海道札幌市中央区旭ケ丘
2,0600041,北海道札幌市中央区大通東
3,0600042,北海道札幌市中央区大通西（１〜１９丁目）
4,0640820,北海道札幌市中央区大通西（２０〜２８丁目）


In [13]:
zip_locations = pd.merge(df_zips2, df_locations2, how='outer', on='full_name')
zip_locations.head(10)

,郵便番号,full_name,緯度,経度
0,0600000,北海道札幌市中央区以下に掲載がない場合,NaN,NaN
1,0640941,北海道札幌市中央区旭ケ丘,NaN,NaN
2,0600041,北海道札幌市中央区大通東,NaN,NaN
3,0600042,北海道札幌市中央区大通西（１〜１９丁目）,NaN,NaN
4,0640820,北海道札幌市中央区大通西（２０〜２８丁目）,NaN,NaN
5,0600031,北海道札幌市中央区北一条東,NaN,NaN
6,0600001,北海道札幌市中央区北一条西（１〜１９丁目）,NaN,NaN
7,0640821,北海道札幌市中央区北一条西（２０〜２８丁目）,NaN,NaN
8,0600032,北海道札幌市中央区北二条東,NaN,NaN
9,0600002,北海道札幌市中央区北二条西（１〜１９丁目）,NaN,NaN


In [14]:
zip_locations.isna().sum()

郵便番号         135181
full_name         0
緯度            51469
経度            51469
dtype: int64

In [15]:
i = 0
no_data = 0
with tqdm(total=len(list(zip_locations.iterrows()))) as pbar:
  for index, row in zip_locations.iterrows():
      pbar.update(1)
      i += 1
      # if i > 100:
      #    break
      if not pd.isnull(row['緯度']):
          next
      # print(row['full_name'], row['緯度'])
      name = row['full_name'].replace('以下に掲載がない場合', '')
      name = re.sub(r'（.+）', '', name)    
      locs = df_locations2[df_locations2['full_name'].str.startswith(name)]
      if locs.size == 0:
        no_data += 1
        # print(row['full_name'], name)
      else:
        ## print(row['full_name'])
        target = locs[0:1]
        ## print(target['緯度'])
        zip_locations.iloc[index, 2] = target['緯度']
        zip_locations.iloc[index, 3]  = target['経度']
print(i, no_data)

100%|█████████████████████████████████████████████████████████████████████████████████████| 328662/328662 [18:39:17<00:00,  4.89it/s]

328662 27255


In [16]:
zip_locations.head(10)

,郵便番号,full_name,緯度,経度
0,0600000,北海道札幌市中央区以下に掲載がない場合,43.042230,141.319722
1,0640941,北海道札幌市中央区旭ケ丘,43.042230,141.319722
2,0600041,北海道札幌市中央区大通東,43.064148,141.372048
3,0600042,北海道札幌市中央区大通西（１〜１９丁目）,43.059131,141.341914
4,0640820,北海道札幌市中央区大通西（２０〜２８丁目）,43.059131,141.341914
5,0600031,北海道札幌市中央区北一条東,43.065394,141.372054
6,0600001,北海道札幌市中央区北一条西（１〜１９丁目）,43.060951,141.341495
7,0640821,北海道札幌市中央区北一条西（２０〜２８丁目）,43.060951,141.341495
8,0600032,北海道札幌市中央区北二条東,43.066575,141.371755
9,0600002,北海道札幌市中央区北二条西（１〜１９丁目）,43.062396,141.341152


In [17]:
zip_locations.isna().sum()

郵便番号         135181
full_name         0
緯度            27255
経度            27255
dtype: int64

In [18]:
zip_locations.to_csv('csvs/zip_locations.csv')